## Feature Engineering: Numerical Features
**Feature engineering for numerical features** consists of:
- [sklearn-Based Transformers](#sklearn-Based-Transformers)
    - `StandardScaler_DF`
    - `RobusScaler_DF`    
    - `MinMaxScaler_DF`
    - `MaxAbsScaler_DF`
    - `Normalizer_DF`
    - `PowerTransformer_DF`
    - `Binarizer_DT`
    - `QuantileTransformer_DF`     
    - `KBinsDiscretizer_DF`
    
- [numpy-Based Transformers](#numpy-Based-Transformers)
    - `Log1pTransformer`
    - `SqrtTransformer`
    - `ReciprocalTransformer`

- [Utility Transformers](#Utility-Transformers)
    - `FeatureUnion_DF`
        - Concatenate all returns of Custom Transformers in dataframe (df).
    - `UniversalTransformer`
        - Transform a given df with a general function a user provides.
    - `PassTransformer`
        - Pass a given df to next without any transformation.
    - `FeatureSelector`
        - Select given features from a df.
    - `FeatureSelector_NUM`
        - Select NUMERICAL features from a df.
    - `FeatureSelector_CAT`
        - Select CATEGORICAL features from a df.
  
- **Note**: 
    - Data will have ***'pandas dataframe'*** format before/after transformation.
    - ***sklearn/numpy-based transfomers*** have the same functionalities in dataframe as their alternatives in sklearn/numpy.

- References: 
    - sklearn Preprocessing: https://scikit-learn.org/stable/modules/preprocessing.html
    - sklear Pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
    - pandas-pipelines-custom-transformers: https://github.com/jem1031/pandas-pipelines-custom-transformers
    - In-house Code: 'data_procesing_for_modeling.py' 
    - Featuretools: https://docs.featuretools.com/#
    - Feature Engine: https://pypi.org/project/feature-engine/
    - Category Encoders: http://contrib.scikit-learn.org/categorical-encoding/    

## Things You Need to Do
### Data: Your Own Pre-Processed TRAIN/TEST
- Create Your Own TRAIN/TEST datasets from a master churn data file.
- Pre-process Your Own TRAIN/TEST datasets using **custom pre-processing transformers**.

### 0. Pre-Process Sample Data

In [1]:
# ### 0. Import Required Packages
# import pandas as pd
# import numpy as np
# import pickle
# import matplotlib.pyplot as plt
# import seaborn as sns
# import datetime as dt
# %matplotlib inline

# # Import Custom Transformers Here!!!
# import PreProcessing_Custom_Transformers as PP           ### Import PreProcessing Customer Transformers

# # Use the Updated Attribute/Imputation Dictionaries!!!
# %run 'data_new/attribute_dictionary.py'
# %run 'data_new/imputation_dictionary.py'

In [2]:
# df_train = pd.read_pickle('data_new/df_train_Fios ONT Competitive Area.pkl')
# df_test  = pd.read_pickle('data_new/df_test_Fios ONT Competitive Area.pkl')

# # Use '' as index
# df_train.set_index('chc_id', inplace=True)
# df_test.set_index('chc_id', inplace=True)

# # TRAIN
# train_X = df_train.drop('status', axis=1).copy()
# train_y = df_train['status']

# # TEST
# test_X  = df_test.drop('status', axis=1).copy()
# test_y  = df_test['status']

# # Sample Size
# print('*'*50 + '\nTRAIN vs TEST Datasets\n' + '*'*50)
# print('Competitive Area: ', df_train.competitive_area.unique())
# print('The Shape of TRAIN Data: ' + str(df_train.shape))
# print('The Shape of TEST Data:  ' + str(df_test.shape))

# ## Churn Rate by Sample Type
# print('\n' + '*'*50 + '\nOverall Churn Rate\n' + '*'*50)
# print('TRAIN: ', df_train.status.value_counts(normalize=True)[1].round(4))
# print('TEST:  ', df_test.status.value_counts(normalize=True)[1].round(4), '\n')

In [3]:
# %%time
# from sklearn.pipeline import Pipeline

# # (1) Make a Pipeline and Instantiate
# PP_Pipe = Pipeline([
#                     ('DataType', PP.Use_DefaultDataType(default_dtypes=attribute_dict)),
#                     ('Missing', PP.Remove_MissingFeatures(missing_threshold=0.99)), 
#                     ('Constant', PP.Remove_ConstantFeatures(unique_threshold=1, missing_threshold=0.00)), 
#                     ('Correlated1', PP.Remove_CorrelatedFeatures(correlation_threshold=0.99)), 
#                     ('Duplicate', PP.Remove_DuplicateFeatures()),
#                     ('Imputer', PP.Use_DefaultImputer(default_imputers=attribute_imputer_dict, default_dtypes=attribute_dict)),
#                     ('Correlated2', PP.Remove_CorrelatedFeatures(correlation_threshold=0.90))
#                   ])
                    

# # (2) fit()
# # default: y=None
# PP_Pipe.fit(train_X, train_y)


# # (3) transform()
# train_X_Preprocessed = PP_Pipe.transform(train_X)
# test_X_Preprocessed  = PP_Pipe.transform(test_X)

# # Feature Dimension
# print('\n' + '*'*50 + '\nBefore vs After Transformation\n' + '*'*50)
# print('TRAIN: Before Transformation:' + str(train_X.shape))
# print('TRAIN: After Transformation: ' + str(train_X_Preprocessed.shape))
# print('TEST:  After Transformation: ' + str(test_X_Preprocessed.shape))


# # (4) Save Data in pickle
# train_processed = pd.concat([train_y, train_X_Preprocessed], axis=1)
# test_processed  = pd.concat([test_y, test_X_Preprocessed], axis=1)

# train_processed.to_pickle('data_new/train_processed.pkl')
# test_processed.to_pickle('data_new/test_processed.pkl')

In [4]:
# **************************************************
# Pre-Processing Results
# **************************************************

# 26 features with greater than 99.0% missing values
# 27 features with 1 or fewer unique value(s)
# 44 features with abs(correlation ) > 0.99 with other features
# 3 features with duplicate columns
# 132 features with abs(correlation ) > 0.9 with other features

# **************************************************
# Before vs After Transformation
# **************************************************
# TRAIN: Before Transformation:(126928, 1016)
# TRAIN: After Transformation: (126928, 770)
# TEST:  After Transformation: (123820, 770)
#
# Machine type
# n1-highmem-32 (32 vCPUs, 208 GB memory)
# CPU times: user 1h 2min 14s, sys: 19min 31s, total: 1h 21min 46s
# Wall time: 1h 19min 40s

### 1. Use Pre-Processed Sample Data

In [5]:
### Import Base Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

### Import Feature Engineering Customer Transformers
import FeatureEngineering_Custom_Transformers as FE

## Remove DataConversionWarning
import warnings
warnings.simplefilter('ignore')
# from sklearn.exceptions import DataConversionWarning
# warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [6]:
# Use Pre-Processed Data as TRAIN and TEST
df_train = pd.read_pickle('data_new/train_processed.pkl')
df_test  = pd.read_pickle('data_new/test_processed.pkl')

# TRAIN
train_X = df_train.drop('status', axis=1).copy()
train_y = df_train['status']

# TEST
test_X  = df_test.drop('status', axis=1).copy()
test_y  = df_test['status']

# Sample Size
print('*'*50 + '\nTRAIN vs TEST Datasets\n' + '*'*50)
# print('Competitive Area: ', df_train.competitive_area.unique())
print('The Shape of TRAIN Data: ' + str(df_train.shape))
print('The Shape of TEST Data:  ' + str(df_test.shape))

## Churn Rate by Sample Type
print('\n' + '*'*50 + '\nOverall Churn Rate\n' + '*'*50)
print('TRAIN: ', df_train.status.value_counts(normalize=True)[1].round(4))
print('TEST:  ', df_test.status.value_counts(normalize=True)[1].round(4), '\n')

**************************************************
TRAIN vs TEST Datasets
**************************************************
The Shape of TRAIN Data: (126928, 771)
The Shape of TEST Data:  (123820, 771)

**************************************************
Overall Churn Rate
**************************************************
TRAIN:  0.0199
TEST:   0.0258 



### sklearn-Based Transformers
We use **sklearn-based custom transfomers** as follows:
- Use a utility transformer `FeatureSelector_NUM` to select numerical features.
   - Use `FeatureSelector_CAT` for categorical feature selection.
   - Use `FeatureSelector` with a feature list for any feature selection.

- Use an appropriate **sklearn-based transfomer(s)** such as `StandardScaler_DF`.
    - Can change feature names with ***prefix*** and ***suffix*** parameters.
    - Return ***pandas dataframe*** with feature names and indices.

In [7]:
### StandardScaler_DF Returning Dataframe
Pipe_Standard    = Pipeline([
                             ('Selector', FE.FeatureSelector_NUM()),
                             ('Standard', FE.StandardScaler_DF(prefix='Standard'))
                           ])
Pipe_Standard.fit(train_X)
train_X_Standard = Pipe_Standard.transform(train_X)
print('\nStandardScaler_DF Returns ', type(train_X_Standard))
train_X_Standard.describe().T.head()


StandardScaler_DF Returns  <class 'pandas.core.frame.DataFrame'>


,count,mean,std,min,25%,50%,75%,max
Standard_csc_m2,126928.0,-1.482161e-14,1.000004,-0.364205,-0.364205,-0.364205,-0.364205,31.125190
Standard_csc_m6,126928.0,-1.326284e-15,1.000004,-0.345107,-0.345107,-0.345107,-0.345107,20.715704
Standard_outage_ov_m4,126928.0,6.855309e-16,1.000004,-0.078335,-0.078335,-0.078335,-0.078335,25.286199
Standard_ool_equip_revenue_m1,126928.0,1.233371e-15,1.000004,-4.241374,0.334360,0.334360,0.334360,7.290401
Standard_ttl_tsr_wait_time_m3,126928.0,2.951832e-15,1.000004,-0.108371,-0.108371,-0.108371,-0.108371,47.122360


In [8]:
### StandardScaler Returning n-dimensional array
Pipe_Standard_np    = Pipeline([
                                ('Selector', FE.FeatureSelector_NUM()),
                                ('Standard', FE.StandardScaler())
                              ])
Pipe_Standard_np.fit(train_X)
train_X_Standard_np = Pipe_Standard_np.transform(train_X)
print('\nStandardScaler Returns ', type(train_X_Standard_np))
pd.DataFrame(train_X_Standard_np).describe().T.head()


StandardScaler Returns  <class 'numpy.ndarray'>


,count,mean,std,min,25%,50%,75%,max
0,126928.0,-1.482161e-14,1.000004,-0.364205,-0.364205,-0.364205,-0.364205,31.125190
1,126928.0,-1.326284e-15,1.000004,-0.345107,-0.345107,-0.345107,-0.345107,20.715704
2,126928.0,6.855309e-16,1.000004,-0.078335,-0.078335,-0.078335,-0.078335,25.286199
3,126928.0,1.233371e-15,1.000004,-4.241374,0.334360,0.334360,0.334360,7.290401
4,126928.0,2.951832e-15,1.000004,-0.108371,-0.108371,-0.108371,-0.108371,47.122360


In [9]:
### RobustScaler_DF
Pipe_Robust    = Pipeline([
                           ('Selector', FE.FeatureSelector_NUM()),
                           ('Robust', FE.RobustScaler_DF(prefix='Robust'))
                         ])
Pipe_Robust.fit(train_X)
train_X_Robust = Pipe_Robust.transform(train_X)
train_X_Robust.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Robust_csc_m2,126928.0,0.266017,0.730408,0.0,0.0,0.0,0.0,23.00
Robust_csc_m6,126928.0,0.245793,0.712226,0.0,0.0,0.0,0.0,15.00
Robust_outage_ov_m4,126928.0,0.006177,0.078851,0.0,0.0,0.0,0.0,2.00
Robust_ool_equip_revenue_m1,126928.0,-0.723418,2.163596,-9.9,0.0,0.0,0.0,15.05
Robust_ttl_tsr_wait_time_m3,126928.0,16.958662,156.487714,0.0,0.0,0.0,0.0,7391.00


In [10]:
### QuantileTransformer_DF
Pipe_Quantile    = Pipeline([
                             ('Selector', FE.FeatureSelector_NUM()),
                             ('Quantile', FE.QuantileTransformer_DF(prefix='Quantile', n_quantiles=100, random_state=0))
                           ])
Pipe_Quantile.fit(train_X)
train_X_Quantile = Pipe_Quantile.transform(train_X)
train_X_Quantile.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Quantile_csc_m2,126928.0,0.158989,0.346047,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.0
Quantile_csc_m6,126928.0,0.145548,0.336131,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.0
Quantile_outage_ov_m4,126928.0,0.006107,0.077709,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.0
Quantile_ool_equip_revenue_m1,126928.0,0.498218,0.203134,1.000000e-07,5.707071e-01,5.707071e-01,5.707071e-01,1.0
Quantile_ttl_tsr_wait_time_m3,126928.0,0.067564,0.246110,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.0


### numpy-Based Transformers
We use **numpy-based custom transfomers** as follows:
- Use a utility transformer `FeatureSelector_NUM` to select numerical features.
- Use an appropriate **numpy-based transfomer(s)** such as `Log1pTransformer`.
    - Can change feature names with ***prefix*** and ***suffix*** parameters.
    - Return ***pandas dataframe*** with feature names and indices.
- Use `UniversalTransformer` to apply a general function a user provides.

In [11]:
### Log1pTransformer
Pipe_Log1p    = Pipeline([
                          ('Selector', FE.FeatureSelector_NUM()),
                          ('Log1p', FE.Log1pTransformer(prefix='Log1p'))
                        ])
Pipe_Log1p.fit(train_X)
train_X_Log1p = Pipe_Log1p.transform(train_X)
train_X_Log1p.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Log1p_csc_m2,126928.0,0.151905,0.353604,0.0,0.000000,0.000000,0.000000,3.178054
Log1p_csc_m6,126928.0,0.139257,0.343807,0.0,0.000000,0.000000,0.000000,2.772589
Log1p_outage_ov_m4,126928.0,0.004270,0.054398,0.0,0.000000,0.000000,0.000000,1.098612
Log1p_ool_equip_revenue_m1,126928.0,1.484394,0.700518,0.0,1.783391,1.783391,1.783391,3.044522
Log1p_ttl_tsr_wait_time_m3,126928.0,0.251926,1.076799,0.0,0.000000,0.000000,0.000000,8.908154


In [12]:
### UniversalTransformer with numpy.log1p function
Pipe_Universal    = Pipeline([
                              ('Selector', FE.FeatureSelector_NUM()),
                              ('Universal', FE.UniversalTransformer(prefix='Universal', function = np.log1p))
                            ])
Pipe_Universal.fit(train_X)
train_X_Universal = Pipe_Universal.transform(train_X)
train_X_Universal.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Universal_csc_m2,126928.0,0.151905,0.353604,0.0,0.000000,0.000000,0.000000,3.178054
Universal_csc_m6,126928.0,0.139257,0.343807,0.0,0.000000,0.000000,0.000000,2.772589
Universal_outage_ov_m4,126928.0,0.004270,0.054398,0.0,0.000000,0.000000,0.000000,1.098612
Universal_ool_equip_revenue_m1,126928.0,1.484394,0.700518,0.0,1.783391,1.783391,1.783391,3.044522
Universal_ttl_tsr_wait_time_m3,126928.0,0.251926,1.076799,0.0,0.000000,0.000000,0.000000,8.908154


In [13]:
### SqrtTransformer
Pipe_Sqrt    = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('Sqrt', FE.SqrtTransformer(prefix='Sqrt'))
                        ])
Pipe_Sqrt.fit(train_X)
train_X_Sqrt = Pipe_Sqrt.transform(train_X)
train_X_Sqrt.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Sqrt_csc_m2,126928.0,0.207482,0.472197,0.0,0.00000,0.00000,0.00000,4.795832
Sqrt_csc_m6,126928.0,0.189690,0.458053,0.0,0.00000,0.00000,0.00000,3.872983
Sqrt_outage_ov_m4,126928.0,0.006154,0.078351,0.0,0.00000,0.00000,0.00000,1.414214
Sqrt_ool_equip_revenue_m1,126928.0,1.857303,0.881595,0.0,2.22486,2.22486,2.22486,4.472136
Sqrt_ttl_tsr_wait_time_m3,126928.0,0.724542,4.053866,0.0,0.00000,0.00000,0.00000,85.970925


In [14]:
### ReciprocalTransformer
Pipe_Reciprocal    = Pipeline([
                               ('Selector', FE.FeatureSelector_NUM()),
                               ('Reciprocal', FE.ReciprocalTransformer(prefix='Reciprocal'))
                             ])
Pipe_Reciprocal.fit(train_X)
train_X_Reciprocal = Pipe_Reciprocal.transform(train_X)
train_X_Reciprocal.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Reciprocal_csc_m2,126928.0,0.143376,0.332278,0.00000,0.00000,0.00000,0.00000,1.00000
Reciprocal_csc_m6,126928.0,0.128130,0.315883,0.00000,0.00000,0.00000,0.00000,1.00000
Reciprocal_outage_ov_m4,126928.0,0.006118,0.077913,0.00000,0.00000,0.00000,0.00000,1.00000
Reciprocal_ool_equip_revenue_m1,126928.0,0.162694,0.077963,-0.20202,0.20202,0.20202,0.20202,0.20202
Reciprocal_ttl_tsr_wait_time_m3,126928.0,0.013553,0.093915,0.00000,0.00000,0.00000,0.00000,1.00000


### Wrap-up: Create a Meta Custom Transfomer for Feature Engineering
- Use `make_pipeline` and `FeatureUnion_DF` to integrate all custom transformers for feature engineering.

#### Instantiate All Other Transformers

In [15]:
### PassTransformer Returning the Raw Numerical Features 
Pipe_Original    = Pipeline([
                             ('Selector', FE.FeatureSelector_NUM()),
                             ('Original', FE.PassTransformer(prefix='Original'))
                           ])
Pipe_Original.fit(train_X)
train_X_Original = Pipe_Original.transform(train_X)
train_X_Original.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Original_csc_m2,126928.0,0.266017,0.730408,0.00,0.00,0.00,0.00,23.0
Original_csc_m6,126928.0,0.245793,0.712226,0.00,0.00,0.00,0.00,15.0
Original_outage_ov_m4,126928.0,0.006177,0.078851,0.00,0.00,0.00,0.00,2.0
Original_ool_equip_revenue_m1,126928.0,4.226582,2.163596,-4.95,4.95,4.95,4.95,20.0
Original_ttl_tsr_wait_time_m3,126928.0,16.958662,156.487714,0.00,0.00,0.00,0.00,7391.0


In [16]:
# ### PowerTransformer_DF: Yeo-Johnson Transformation
# Pipe_YJ = Pipeline([
#                          ('Selector', FE.FeatureSelector_NUM()),
#                          ('YJ', FE.PowerTransformer_DF(prefix='YJ', method='yeo-johnson'))
#                   ])

### Binarize
Pipe_Binary = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('Binary', FE.Binarizer_DF(prefix='Binary', threshold=0))
                      ])

### MinMaxScaler
Pipe_MinMax = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('MinMax', FE.MinMaxScaler_DF(prefix='MinMax', feature_range=(0, 1)))
                      ])

### MaxAbsScaler
Pipe_MaxAbs = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('MinMax', FE.MaxAbsScaler_DF(prefix='MaxAbs'))
                      ])


### Normalizer
Pipe_Norm = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('Norm', FE.Normalizer_DF(prefix='Norm'))
                    ])

### KBinsDiscretizer
Pipe_KBin = Pipeline([
                         ('Selector', FE.FeatureSelector_NUM()),
                         ('KBin', FE.KBinsDiscretizer_DF(prefix='KBin', n_bins=10))
                    ])

In [17]:
%%time
warnings.simplefilter('ignore')
    
# (1) Make a Pipeline in Parallel and Instantiate 
FE_Pipe    = FE.FeatureUnion_DF([
                                 ('Original_Pipe', Pipe_Original),     
                                 ('Standard_Pipe', Pipe_Standard), 
                                 ('Robst_Pipe', Pipe_Robust), 
                                 ('MinMax_Pipe', Pipe_MinMax), 
                                 ('MaxAbs_Pipe', Pipe_MaxAbs),                                                                                                                                
                                 ('Norm_Pipe', Pipe_Norm), 
#                                  ('YJ_Pipe', Pipe_YJ), 
                                 ('Quantile_Pipe', Pipe_Quantile), 
                                 ('KBin', Pipe_KBin),     
                                 ('Binary_Pipe', Pipe_Binary),     
                                 ('Log1p_Pipe', Pipe_Log1p), 
                                 ('Sqrt_Pipe', Pipe_Sqrt), 
                                 ('Reciprocal_Pipe', Pipe_Reciprocal)                                                                                                                                  
                               ])

# (2) fit()
FE_Pipe.fit(train_X, train_y)

# (3) transform()
train_X_FE = FE_Pipe.transform(train_X)
test_X_FE  = FE_Pipe.transform(test_X)

# Feature Dimension
print('\n' + '*'*50 + '\nBefore vs After Feature Engineering (FE)\n' + '*'*50)
print('TRAIN: Before FE:' + str(train_X_Original.shape))
print('TRAIN: After FE: ' + str(train_X_FE.shape))
print('TEST:  After FE: ' + str(test_X_FE.shape))


**************************************************
Before vs After Feature Engineering (FE)
**************************************************
TRAIN: Before FE:(126928, 420)
TRAIN: After FE: (126928, 5040)
TEST:  After FE: (123820, 5040)
CPU times: user 3min 5s, sys: 3min 54s, total: 6min 59s
Wall time: 2min 7s


In [18]:
p_list     = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, .6, .7, .8, .9, .95, .99]
corr1      = train_X_Original.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr2      = train_X_Standard.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr3      = train_X_Robust.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr4      = train_X_Quantile.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr5      = train_X_Log1p.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr6      = train_X_Sqrt.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
corr7      = train_X_Reciprocal.apply(lambda x: x.corr(train_y)).to_frame().describe(percentiles=p_list)
train_corr = pd.concat([corr1, corr2, corr3, corr4, corr5, corr6, corr7], axis=1)
train_corr.columns = ['Original', 'Standard', 'Robust', 'Quantile', 'Log1p', 'Sqrt', 'Reciprocal']
train_corr

,Original,Standard,Robust,Quantile,Log1p,Sqrt,Reciprocal
count,410.000000,410.000000,410.000000,420.000000,379.000000,409.000000,410.000000
mean,0.000685,0.000685,0.000685,0.000801,0.000820,0.001175,-0.000150
std,0.007931,0.007931,0.007931,0.008322,0.008506,0.009323,0.005098
min,-0.041030,-0.041030,-0.041030,-0.036014,-0.032108,-0.047582,-0.020818
1%,-0.020200,-0.020200,-0.020200,-0.022765,-0.020343,-0.021939,-0.016697
5%,-0.010883,-0.010883,-0.010883,-0.011794,-0.011897,-0.012420,-0.007028
10%,-0.005972,-0.005972,-0.005972,-0.006473,-0.006404,-0.006511,-0.005354
20%,-0.003819,-0.003819,-0.003819,-0.003792,-0.004126,-0.004087,-0.003332
30%,-0.002374,-0.002374,-0.002374,-0.002121,-0.002824,-0.002528,-0.002094
40%,-0.001097,-0.001097,-0.001097,-0.000826,-0.001128,-0.001060,-0.000939
